In [71]:
import glob
import pandas as pd
import os
import shutil

In [8]:
files = glob.glob('I:/ROV/*.srv8')

In [9]:
df =pd.DataFrame({'files':files})

In [10]:
df['basename']=df.files.apply(os.path.basename)

In [12]:
df['basename'].str.split('_',expand=True)

,0,1,2
0,Divelog,20220518,162518.srv8
1,Divelog,20220518,162126.srv8
2,Divelog,20220517,170017.srv8
3,Divelog,20220517,170010.srv8
4,Divelog,20220517,170007.srv8
...,...,...,...
144,Divelog,20220710,105735.srv8
145,Divelog,20220710,103056.srv8
146,Divelog,20220710,100530.srv8
147,Divelog,20220710,090736.srv8


In [29]:
df['starttime']=pd.to_datetime(df['basename'].apply(os.path.splitext).apply(lambda x:x[0]),format='Divelog_%Y%m%d_%H%M%S')

In [32]:
df['date']=df['starttime'].dt.date

In [37]:
df['tow']=1
df['tow']=df.groupby('date')['tow'].cumsum()

In [38]:
df['tow']

0      1
1      2
2      1
3      2
4      3
      ..
144    2
145    3
146    4
147    5
148    6
Name: tow, Length: 149, dtype: int64

In [43]:
df['path'] =df['date'].astype(str)+'/'+df['tow']+df['basename']

TypeError: can only concatenate str (not "int") to str

In [65]:
f"{x.date}/{x.tow:03d}/{x.basename}"

'2022-05-18/001/Divelog_20220518_162518.srv8'

In [52]:
x=df.iloc[0]

In [73]:
df['dest']=df.apply(lambda x:f"I:ROV/{x.date}/{x.tow:03d}",axis=1)

In [74]:
df

,files,basename,starttime,date,tow,dest
0,I:/ROV\Divelog_20220518_162518.srv8,Divelog_20220518_162518.srv8,2022-05-18 16:25:18,2022-05-18,1,I:ROV/2022-05-18/001
1,I:/ROV\Divelog_20220518_162126.srv8,Divelog_20220518_162126.srv8,2022-05-18 16:21:26,2022-05-18,2,I:ROV/2022-05-18/002
2,I:/ROV\Divelog_20220517_170017.srv8,Divelog_20220517_170017.srv8,2022-05-17 17:00:17,2022-05-17,1,I:ROV/2022-05-17/001
3,I:/ROV\Divelog_20220517_170010.srv8,Divelog_20220517_170010.srv8,2022-05-17 17:00:10,2022-05-17,2,I:ROV/2022-05-17/002
4,I:/ROV\Divelog_20220517_170007.srv8,Divelog_20220517_170007.srv8,2022-05-17 17:00:07,2022-05-17,3,I:ROV/2022-05-17/003
...,...,...,...,...,...,...
144,I:/ROV\Divelog_20220710_105735.srv8,Divelog_20220710_105735.srv8,2022-07-10 10:57:35,2022-07-10,2,I:ROV/2022-07-10/002
145,I:/ROV\Divelog_20220710_103056.srv8,Divelog_20220710_103056.srv8,2022-07-10 10:30:56,2022-07-10,3,I:ROV/2022-07-10/003
146,I:/ROV\Divelog_20220710_100530.srv8,Divelog_20220710_100530.srv8,2022-07-10 10:05:30,2022-07-10,4,I:ROV/2022-07-10/004
147,I:/ROV\Divelog_20220710_090736.srv8,Divelog_20220710_090736.srv8,2022-07-10 09:07:36,2022-07-10,5,I:ROV/2022-07-10/005


In [75]:
for index,item in df.iterrows():
    os.makedirs(item.dest,exist_ok=True)
    shutil.move(item.files,f'{item.dest}/{item.basename}')